In [13]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [14]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/eo_telesat_post_processed.h5"

row_limit = -1

store = pd.HDFStore(file_path)

instances = store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances = instances.sort_index(0)

In [19]:
plt.close('all')

case, target = pickle.load(open('raw/eo_settings.pkl', 'rb'))

T_orbit = case['T_orbit_s']
T_sim = case['T_sim_s']

fig = plt.figure(figsize=(3.2, 2.4))

for i, pass_df in instances.groupby(level=0):
    tof = pass_df.tof
    d = pass_df.d/1000
    closer_too_polar = np.argmin(np.abs(d.to_numpy()[0] - np.array([1400, 800])))
    if closer_too_polar:
        line, = plt.plot(tof, d, linewidth=0.35, color='tab:blue')
    else:
        line, = plt.plot(tof, d, linewidth=0.35, color='tab:red')

print(i)

plt.xlabel('Time of flight [s]')
plt.ylabel('Range [km]')
plt.xlim((0, T_sim))

plt.grid()
PlotUtil.apply_report_formatting()
fig.set_size_inches(3.2*2, 2.4, forward=True)

plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_range.pdf')
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_range.svg')

In [16]:
store.close()